In [ ]:
import torch
from torchvision import models
import sys
sys.path.append('../')
from pyqcr.torchgraph.tracer import TorchTracer
from pyqcr.quantization import compatability as comp

In [ ]:
model = models.resnet18()
inp = torch.rand((1,3,224,224))
with TorchTracer() as tt:
    tt.trace_model(model, inp)
    model.graph = tt.to_graph()

model.graph.to_namegraph()

In [ ]:
# Find all modules which used multiple times during forward. 
# If there is reused modules pytorch1.6 quantization won't work correctly.
comp.get_reused_modules(model)

In [ ]:
# In order to quantize pytorch model with 1.6 API we need to convert all non-module operations to modules.
comp.get_non_module_ops(model)